In [13]:
from classifier_lewidi import *
from params_lewidi import *
from DataManager import *

import os
import argparse
from datetime import datetime
import pytz
import re


In [2]:


filepath1 = '../data/ConvAbuse_dataset/'
filepath2 = '../data/ArMIS_dataset/'
filepath3 = '../data/HS-Brexit_dataset/'
filepath4 = '../data/MD-Agreement_dataset/'




In [14]:

data = DataManager(filepath1, filepath2, filepath3, filepath4)

I'm in with the conv_train dataset


KeyboardInterrupt: 

In [8]:
def_params = params_lewidi()

def_params.task = "single"

In [15]:

key = "conv_train"

annotators = list(sorted(set(itertools.chain.from_iterable(data.dataset_groups[key]['annotators'].str.findall("\w+")))))
    
model = ToxicityClassifier(data.dataset_groups[key], annotators=annotators, params=def_params)



431
Data shape after majority voting (2398, 16)
[('batch_size', 4), ('learning_rate', 1e-07), ('max_len', 128), ('num_epochs', 3), ('random_state', 9999), ('num_folds', 5), ('task', 'single'), ('batch_weight', None), ('sort_by', None), ('stratified', True), ('predict', 'label'), ('mc_passes', 10)]


In [16]:
score, results = model.CV()

score["params"] = ", ".join(key + ": " + str(val) for key, val in def_params.__dict__.items())
score["task"] = model.params.task


Fold # 1


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


TypeError: compute_class_weight() takes 1 positional argument but 3 were given

In [ ]:

pacific = pytz.timezone('US/Pacific')
sa_time = datetime.now(pacific)
name_time = sa_time.strftime('%m%d%y-%H:%M')
score["time"] = name_time
print(score)

score_dir = os.path.join(def_params.source_dir, "results", "GHC", "classification.csv")
result_dir = os.path.join(def_params.source_dir, "results", "GHC", name_time + "_" + def_params.task + ".csv")
if os.path.exists(score_dir):
    pd.DataFrame.from_records([score]).to_csv(score_dir, header=False,  index=False, mode="a")
else:
    pd.DataFrame.from_records([score]).to_csv(score_dir, index=False)

    pd.DataFrame.from_dict(results).to_csv(result_dir, index=False)
